In [1]:
import pandas as pd
import numpy as np
import pickle
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import time
import argparse
import os
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.init as init
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import wandb
from tqdm import tqdm


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
wandb.login() #9a0ae44e5144f21ddaf2dbb8139e4f300be33f43

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zihaom96. Use `wandb login --relogin` to force relogin


True

In [3]:
df = pd.read_csv('../baselines/RNN/datasets/Dataset 1.csv') #to be comparable with the size of other datasets, select ~5000 data points from the reference
df['TRIMER_mol'] = df['TRIMER'].apply(Chem.MolFromSmiles)
df = df.dropna()
#df = df.head(5000)

In [4]:
nbits = 1024
fp = df['TRIMER_mol'].apply(lambda m: AllChem.GetMorganFingerprintAsBitVect(m, radius=3, nBits=nbits))
fp_array = np.asarray([np.array(fp[i]) for i in range(len(fp))])
df['fps'] = fp

In [5]:
Mix_X_100Block = np.repeat(fp_array, 100, axis=0)
Mix_X_100Block = Mix_X_100Block.reshape(len(df), 100, nbits)


In [6]:

target_column = "IP (eV)" #"EA (eV)"

X_train, X_test, y_train, y_test = train_test_split(Mix_X_100Block, df[target_column], test_size=0.2, random_state=11)


In [7]:
X_train_tensor = torch.tensor(X_train, dtype=torch.long)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [8]:
class MolecularDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        
        return x, y

In [9]:
train_dataset = MolecularDataset(X_train_tensor, y_train_tensor)
test_dataset = MolecularDataset(X_test_tensor, y_test_tensor)


In [10]:
type(df['fps'][0])

rdkit.DataStructs.cDataStructs.ExplicitBitVect

In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

#reset index
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [12]:
class TrimerDataset(Dataset):
    def __init__(self, data, target_column):
        self.data = data
        self.target_column = target_column

    def __getitem__(self, index):
        tokens = self.data.loc[index, "fps"]
        target = self.data.loc[index, self.target_column]

        tokens_tensor = torch.tensor(tokens, dtype=torch.long)
        target_tensor = torch.tensor(target, dtype=torch.float32)

        return tokens_tensor, target_tensor

    def __len__(self):
        return len(self.data)


In [13]:
#target_column = "EA (eV)"
# train_dataset = TrimerDataset(train_data, target_column)
# test_dataset = TrimerDataset(test_data, target_column)


In [14]:
batch_size = 128#128
shuffle = True
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)


In [15]:
for i in range(1):
    tokens, target = train_dataset[i]
    print(f"Sample {i + 1}:")
    print(f"Tokens: {tokens}")
    print(f"Target: {target}\n")

/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Sample 1:
Tokens: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Target: 6.040721893310547



In [16]:
# from Veronika
# class RNN(nn.Module):
#     def __init__(self, seq_len, emb_dim, lstm_dim, linear_dim, out_dim, num_tokens=None):
#         super(RNN, self).__init__()
#         self.seq_len = seq_len
#         self.emb = nn.Embedding(num_tokens, emb_dim) if num_tokens is not None else None
#         self.lstm1 = nn.LSTM(emb_dim, lstm_dim, bidirectional=True, batch_first=True)
#         self.lstm2 = nn.LSTM(lstm_dim*2, lstm_dim, bidirectional=True, batch_first=True)
#         self.lstm_dim = lstm_dim

#         self.mlp = nn.Sequential(
#             nn.Linear(lstm_dim * 2, int(lstm_dim/2)),
#             nn.ReLU(),
#         )
        
#         self.last = nn.Linear(int(lstm_dim/2)*seq_len, out_dim)

#     def forward(self, data):  # 2D
#         x = self.emb(data) if self.emb else data
#         #print(x.shape)
#         x, _ = self.lstm1(x)
#         x, _ = self.lstm2(x)
#         # x = x.reshape(x.shape[0], -1)  # Flatten time and batch dims
#         x = self.mlp(x) #removed mlp
#         #x = x.reshape(x.shape[0], -1)  # Flatten time dim into last one (instead of before into sample dim)
#         x = x.reshape(-1, int(self.lstm_dim/2) * self.seq_len)
#         #x = self.mlp(x)
#         x = self.last(x)
#         return x

In [27]:
# adjusted from Veronika
class RNN(nn.Module):
    def __init__(self, seq_len, emb_dim, lstm_dim, linear_dim, out_dim, num_tokens=None):
        super(RNN, self).__init__()
        self.seq_len = seq_len
        self.emb = nn.Embedding(num_tokens, emb_dim) if num_tokens is not None else None
        self.lstm1 = nn.LSTM(emb_dim, lstm_dim, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(lstm_dim*2, lstm_dim, bidirectional=True, batch_first=True)
        self.lstm_dim = lstm_dim

        self.mlp = nn.Sequential(
            nn.Linear(lstm_dim * 2, int(lstm_dim/2)),
            nn.ReLU(),
        )
        init.xavier_uniform_(self.mlp[0].weight)
        init.zeros_(self.mlp[0].bias)

        self.last = nn.Linear(int(lstm_dim/2)*seq_len, out_dim)
        init.xavier_uniform_(self.last.weight)
        init.zeros_(self.last.bias)


    def forward(self, data):  # 2D
        #x = self.emb(data) if self.emb else data
        #print(x.shape)
        x = data
        x, _ = self.lstm1(x)
        print(f"LSTM Layer 1 output shape: {x.shape}")

        x, _ = self.lstm2(x)
        print(f"LSTM Layer 2 output shape: {x.shape}")

        # x = x.reshape(x.shape[0], -1)  # Flatten time and batch dims
        x = self.mlp(x) #removed mlp
        print(f"MLP output shape: {x.shape}")

        #x = x.reshape(x.shape[0], -1)  # Flatten time dim into last one (instead of before into sample dim)
        x = x.reshape(-1, int(self.lstm_dim/2) * self.seq_len)
        print(f"Reshape output shape: {x.shape}")

        #x = self.mlp(x)
        x = self.last(x)
        print(f"Output Layer shape: {x.shape}")

        return x
    
    def __str__(self):
        layers = [
            ('Embedding Layer', self.emb),
            ('LSTM Layer 1', self.lstm1),
            ('LSTM Layer 2', self.lstm2),
            ('MLP', self.mlp),
            ('Output Layer', self.last),
        ]
        layer_str = '\n'.join([f'{name}: {layer}' for name, layer in layers])
        return f'RNN Model:\n{layer_str}'


In [28]:
seq_len = 100
emb_dim = 1024
lstm_dim = 20 #512
linear_dim = 10 #256

out_dim = 1
num_tokens=1024
learning_rate = 0.001

model = RNN(seq_len, emb_dim, lstm_dim, linear_dim, out_dim).to(device)
loss_fn = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
weight_decay = 0.01  # L2 regularization strength
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [21]:
print(model)

RNN Model:
Embedding Layer: None
LSTM Layer 1: LSTM(1024, 20, batch_first=True, bidirectional=True)
LSTM Layer 2: LSTM(40, 20, batch_first=True, bidirectional=True)
MLP: Sequential(
  (0): Linear(in_features=40, out_features=10, bias=True)
  (1): ReLU()
)
Output Layer: Linear(in_features=1000, out_features=1, bias=True)


In [75]:
### For Trimer dataset

# seq_len = 1024
# emb_dim = 100 #50, 100, 200, or 300
# lstm_dim= 20 #50, 100, 200
# linear_dim = 100
# out_dim = 1
# num_tokens=1024
# learning_rate = 0.001

# model = RNN(seq_len, emb_dim, lstm_dim, linear_dim, out_dim, num_tokens).to(device)
# loss_fn = nn.MSELoss()
# #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# weight_decay = 0.01  # L2 regularization strength
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [29]:
def train_RNN(model, train_loader, test_loader, loss_fn, optimizer, num_epochs=10, device="cpu"):
    model.to(device)
    train_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        #for x, y in train_loader:
        for x, y in tqdm(train_loader, desc=f'Training epoch {epoch + 1}/{num_epochs}'):
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            output = model(x)
            loss = loss_fn(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Testing
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)

                output = model(x)
                loss = loss_fn(output, y)
                test_loss += loss.item()

        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        
        #wandb.log({"train_loss": test_loss, "test_losses": test_loss})

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    return train_losses, test_losses

In [80]:
def train_with_wandb():
    # Initialize a new wandb run
    run = wandb.init()
    
     # Get the hyperparameters from the Sweep
    config = wandb.config
    learning_rate = config.learning_rate
    num_epochs = 20#config.num_epochs
    weight_decay = config.weight_decay
    
   # Set the run name with hyperparameter details
    run_name = f"lr_{learning_rate:.5f}_wd_{weight_decay:.5f}"
    wandb.run.name = run_name
    wandb.run.save()

    # Build your model, optimizer, and loss function
    model = RNN(seq_len, emb_dim, lstm_dim, linear_dim, out_dim).to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Train your model using train_RNN function
    train_losses, test_losses = train_RNN(model, train_dataloader, test_dataloader, loss_fn, optimizer, num_epochs, device)
    
    # Finish the run
    run.finish()

In [85]:
sweep_config = {
    "name": "RNN Sweep",
    "method": "random",
    "metric": {
        "name": "test_loss",
        "goal": "minimize",
    },
    "parameters": {
        "learning_rate": {
            "min": 0.00001,
            "max": 0.001,
        },
        "weight_decay": {
            "min": 0.00001,
            "max": 0.1,
        },
    },
}



In [ ]:

sweep_id = wandb.sweep(sweep_config, project="RNN-Pytorch-polymer")
wandb.agent(sweep_id, train_with_wandb, count=10)

Create sweep with ID: l0dl7ho9
Sweep URL: https://wandb.ai/zihaom96/RNN-Pytorch-polymer/sweeps/l0dl7ho9


wandb: Agent Starting Run: k6lj948g with config:
wandb: 	learning_rate: 0.0009399172548070652
wandb: 	weight_decay: 0.05570658805320806


Training epoch 1/20:   0%|          | 0/300 [00:00<?, ?it/s]/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, targ

Epoch 1/20, Train Loss: 1.2785, Test Loss: 0.2433


Training epoch 2/20: 100%|██████████| 300/300 [00:23<00:00, 12.59it/s]


...

Training epoch 5/20: 100%|██████████| 300/300 [00:24<00:00, 12.07it/s]


Epoch 5/20, Train Loss: 1.1592, Test Loss: 1.0763


Training epoch 6/20: 100%|██████████| 300/300 [00:23<00:00, 12.71it/s]


Epoch 6/20, Train Loss: 0.9940, Test Loss: 0.9335


Training epoch 7/20:  86%|████████▌ | 258/300 [00:19<00:03, 11.58it/s]

In [30]:
num_epochs = 1

# wandb.init(
#     # set the wandb project where this run will be logged
#     project="RNN-Pytorch-polymer",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.01,
#     "architecture": "RNN",
#     "dataset": "Trimer",
#     "epochs": num_epochs,
#     }
# )

train_losses, test_losses = train_RNN(model, train_dataloader, test_dataloader, loss_fn, optimizer, num_epochs, device)
#train_losses, test_losses = train_RNN(model, train_dataset, test_dataset, loss_fn, optimizer, num_epochs, device)

Training epoch 1/1:   0%|          | 0/300 [00:00<?, ?it/s]/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, targe

LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])


Training epoch 1/1:   2%|▏         | 6/300 [00:00<00:23, 12.58it/s]

LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])


Training epoch 1/1:   3%|▎         | 8/300 [00:00<00:22, 12.70it/s]

...

LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])


Training epoch 1/1:  99%|█████████▉| 298/300 [00:23<00:00, 12.25it/s]/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([118])) that is different to the input size (torch.Size([118, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Training epoch 1/1: 100%|██████████| 300/300 [00:23<00:00, 12.69it/s]

LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([118, 100, 40])
LSTM Layer 2 output shape: torch.Size([118, 100, 40])
MLP output shape: torch.Size([118, 100, 10])
Reshape output shape: torch.Size([118, 1000])
Output Layer shape: torch.Size([118, 1])


LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([128, 100, 40])
LSTM Layer 2 output shape: torch.Size([128, 100, 40])
MLP output shape: torch.Size([128, 100, 10])
Reshape output shape: torch.Size([128, 1000])
Output Layer shape: torch.Size([128, 1])
LSTM Layer 1 output shape: torch.Size([1

/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([126])) that is different to the input size (torch.Size([126, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
current_time = time.strftime("%Y%m%d-%H%M%S")
model_path = f"models/model_{current_time}.pt"

torch.save(model.state_dict(), model_path)
print("Saved PyTorch Model State to "+model_path)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

def evaluate_model(model, dataloader, device):
    model.eval()
    all_outputs = []
    all_targets = []
    
    with torch.no_grad():
        for batch in dataloader:
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            
            output = model(x)
            all_outputs.extend(output.cpu().numpy())
            all_targets.extend(y.cpu().numpy())

    return np.array(all_outputs), np.array(all_targets)

# Evaluate the model
predicted, targets = evaluate_model(model, test_dataloader, device)

# Calculate R2 score, MAE, and RMSE
r2 = r2_score(targets, predicted)
mae = mean_absolute_error(targets, predicted)
rmse = np.sqrt(mean_squared_error(targets, predicted))

print(f"R2 Score: {r2:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")

In [ ]:
targets_np = np.array(targets).flatten()
predicted_np = np.array(predicted).flatten()

plt.figure(figsize=(10, 10))
plt.scatter(predicted_np, targets_np, alpha=0.5)

# Fit a linear regression model
m, b = np.polyfit(predicted_np, targets_np, 1)

# Create line points based on the min and max of the predicted values
line_x = np.linspace(min(predicted_np), max(predicted_np), 100)
line_y = m * line_x + b

# Plot the best fit line
plt.plot(line_x, line_y, '--', c='r')

plt.xlabel('Predicted Values')
plt.ylabel('True Values')
plt.title('Scatter plot of Predicted vs. True Values')
plt.show()


In [64]:
class RNNModel(nn.Module):
    def __init__(self, input_shape, lstm_units):
        super(RNNModel, self).__init__()
        self.lstm_units = lstm_units
        self.bidir_lstm1 = nn.LSTM(input_shape[1], lstm_units, batch_first=True, bidirectional=True)
        self.bidir_lstm2 = nn.LSTM(lstm_units * 2, lstm_units, batch_first=True, bidirectional=True)
        self.time_distributed_dense = nn.Linear(lstm_units * 2, int(lstm_units / 2))
        self.reshape_dim = int(lstm_units / 2 * input_shape[0])
        self.final_dense = nn.Linear(self.reshape_dim, 1)

    def forward(self, x):
        x, _ = self.bidir_lstm1(x)
        x, _ = self.bidir_lstm2(x)
        x = self.time_distributed_dense(x)
        x = torch.relu(x)
        x = x.view(-1, self.reshape_dim)
        x = self.final_dense(x)
        return x

# Usage example
input_shape = (100, 1024)
lstm_units = 20
model = RNNModel(input_shape, lstm_units)


In [65]:
loss_fn = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
learning_rate = 0.0001
weight_decay = 0.01  # L2 regularization strength
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [66]:
num_epochs = 20
train_losses, test_losses = train_RNN(model, train_dataloader, test_dataloader, loss_fn, optimizer, num_epochs, device)

Training epoch 1/20:   0%|          | 0/125 [00:00<?, ?it/s]/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/zmao_umass_edu/.conda/envs/figo/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, targ

Epoch 1/20, Train Loss: 24.4514, Test Loss: 0.6915


Training epoch 2/20: 100%|██████████| 125/125 [00:06<00:00, 19.94it/s]


Epoch 2/20, Train Loss: 0.4679, Test Loss: 0.3951


Training epoch 3/20: 100%|██████████| 125/125 [00:07<00:00, 15.66it/s]


...

Epoch 5/20, Train Loss: 0.2532, Test Loss: 0.2589


Training epoch 6/20: 100%|██████████| 125/125 [00:06<00:00, 18.33it/s]


Epoch 6/20, Train Loss: 0.2372, Test Loss: 0.2416


Training epoch 7/20:  46%|████▌     | 57/125 [00:05<00:06, 11.24it/s]


KeyboardInterrupt: 

In [ ]:
# Evaluate the model
predicted, targets = evaluate_model(model, test_dataloader, device)

# Calculate R2 score, MAE, and RMSE
r2 = r2_score(targets, predicted)
mae = mean_absolute_error(targets, predicted)
rmse = np.sqrt(mean_squared_error(targets, predicted))

print(f"R2 Score: {r2:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")

In [ ]:
targets_np = np.array(targets).flatten()
predicted_np = np.array(predicted).flatten()

plt.figure(figsize=(10, 10))
plt.scatter(predicted_np, targets_np, alpha=0.5)

# Fit a linear regression model
m, b = np.polyfit(predicted_np, targets_np, 1)

# Create line points based on the min and max of the predicted values
line_x = np.linspace(min(predicted_np), max(predicted_np), 100)
line_y = m * line_x + b

# Plot the best fit line
plt.plot(line_x, line_y, '--', c='r')

plt.xlabel('Predicted Values')
plt.ylabel('True Values')
plt.title('Scatter plot of Predicted vs. True Values')
plt.show()
